<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-13 13:44:30
-------------------
qualified stocks: 91
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  3.50 L
Current:  1.33 C
-------------------
Today PnL: -1.33 L (-0.99%)
Current PnL: -16.87 L (-11.75%)
CY Booked + Current PnL: -10.50 L (-7.31%)
-------------------
Total profit:  2.81 L
Total loss:  -19.67 L
-------------------
Total Booked + Current PnL: 16.48 L (14.14%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.79%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.75 L (54.66%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,827.06,983.0,-5.02,H-LC,87.91,96562.0,11375.0,4683.0,-0.66,13.35,4.85,18.85,35.0,2.43,0.75,28.57,XY25,NTT,INSURANCE
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,82.42,132705.0,-9774.0,10391.0,1.53,-6.86,7.83,0.44,242.0,-0.94,1.02,56.63,OX40N,NTT,BANKS
60,RELIANCE,1291.83,1526.0,4.98,X-LC,36.26,155859.0,15050.0,10474.0,-0.81,10.69,6.72,18.13,17.0,1.44,1.20,23.07,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,-0.40,M-LC,45.05,99755.0,21393.0,12639.0,-2.35,27.30,12.67,43.43,77.0,1.69,0.77,34.07,XY24,NTT,MISC
76,TATAELXSI,7332.28,7332.0,-14.64,X-MC,69.23,89607.0,-13045.0,13038.0,-1.13,-12.71,14.55,-0.00,57.0,-1.00,0.69,36.21,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,561.33,680.00,-16.80,H-MC,49.45,228299.0,960.0,47098.0,-0.85,0.42,20.63,21.14,117.0,0.02,1.76,27.82,XY24,NTT,PAINTS
32,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,60.44,151260.0,347.0,60111.0,-0.94,0.23,39.74,40.06,100.0,0.01,1.17,23.72,AR,ATH,AUTO
88,VOLTAS,1278.28,1929.20,-29.62,H-MC,41.76,191790.0,48.0,97583.0,-0.88,0.03,50.88,50.92,99.0,0.00,1.48,6.23,XY25,ATH,AC
56,PGHH,13388.00,18062.58,-20.14,X-MC,13.19,201540.0,720.0,69390.0,0.26,0.36,34.43,34.92,53.0,0.01,1.56,5.05,X40,ATH,FMCG
19,CIPLA,1495.00,1795.00,-17.66,H-LC,23.08,100674.0,509.0,19591.0,-0.01,0.51,19.46,20.07,29.0,0.03,0.78,8.51,X40N,BTT,PHARMA


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7483.33,M-SC,100.00,83224.0,-10586.0,87202.0,3.53,-11.28,104.78,81.67,238.0,-0.12,0.64,61.17,XR,NTT,CERAMICS
82,UJJIVANSFB,52.77,53.00,52.86,M-SC,82.42,132705.0,-9774.0,10391.0,1.53,-6.86,7.83,0.44,242.0,-0.94,1.02,56.63,OX40N,NTT,BANKS
61,REPCOHOME,515.07,880.00,-54.55,H-SC,52.75,169210.0,-35788.0,181038.0,1.41,-17.46,106.99,70.85,134.0,-0.20,1.31,36.59,XY24,NTT,FINANCE
37,ICICIGI,1839.64,2260.25,-13.82,H-MC,46.15,141999.0,5866.0,25262.0,0.89,4.31,17.79,22.86,34.0,0.23,1.10,17.47,X40,ATH,INSURANCE
70,SIS,477.00,477.00,2225.44,M-SC,84.62,62960.0,-18130.0,18132.0,0.41,-22.36,28.80,0.00,225.0,-1.00,0.49,27.60,OX40N,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,SURYODAY,189.78,240.0,57.89,H-SC,81.32,112417.0,-37509.0,77186.0,-3.57,-25.02,68.66,26.46,143.0,-0.49,0.87,44.01,XR,NTT,BANKS
85,VALIANTORG,512.64,838.0,-389.46,H-SC,98.90,123162.0,-10124.0,94724.0,-3.43,-7.60,76.91,63.47,142.0,-0.11,0.95,106.45,XR,NTT,CHEMICALS
58,RAJESHEXPO,517.65,518.0,1782.58,M-SC,29.67,51368.0,-85809.0,85903.0,-3.38,-62.55,167.23,0.07,232.0,-1.00,0.40,27.75,OX40N,NTT,JEWELLERY
47,KANSAINER,299.63,340.0,-68.77,H-SC,7.69,218250.0,-51417.0,87758.0,-2.94,-19.07,40.21,13.47,144.0,-0.59,1.68,9.24,XY24,NTT,PAINTS
77,TATAMOTORS,761.77,1065.0,-52.64,M-LC,32.97,259672.0,-16851.0,126928.0,-2.86,-6.09,48.88,39.81,69.0,-0.13,2.00,24.45,XY24,NTT,AUTO


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,82.42,132705.0,-9774.0,10391.0,1.53,-6.86,7.83,0.44,242.0,-0.94,1.02,56.63,OX40N,NTT,BANKS
76,TATAELXSI,7332.28,7332.0,-14.64,X-MC,69.23,89607.0,-13045.0,13038.0,-1.13,-12.71,14.55,-0.00,57.0,-1.00,0.69,36.21,OX40N,NTT,IT
74,SYMPHONY,1306.42,1306.0,-29.94,M-SC,2.20,149327.0,-21814.0,21757.0,-2.04,-12.75,14.57,-0.03,190.0,-1.00,1.15,7.33,OX40N,NTT,DURABLES
87,VIPIND,488.80,489.0,-930.51,H-SC,91.21,79102.0,-16214.0,16255.0,-2.66,-17.01,20.55,0.04,153.0,-1.00,0.61,56.02,OX40N,NTT,MISC
18,CERA,8421.60,8422.0,-12.41,X-SC,63.74,90337.0,-19144.0,19151.0,-0.81,-17.49,21.20,0.00,65.0,-1.00,0.70,36.73,OX40N,NTT,CERAMICS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-10.62,X-LC,47.25,286400.0,24799.0,120832.0,-0.53,9.48,42.19,55.67,1.0,0.21,2.21,16.16,X40,BTT,IT
78,TCS,3794.03,4998.00,-17.76,X-LC,21.98,268297.0,-27637.0,121539.0,0.16,-9.34,45.30,31.73,2.0,-0.23,2.07,7.39,X40,BTT,IT
34,HINDUNILVR,2413.81,2723.00,-17.27,X-LC,16.48,259706.0,-10641.0,45267.0,-0.60,-3.94,17.43,12.81,5.0,-0.24,2.00,7.48,XY25,NTT,FMCG
14,BRITANNIA,4983.33,6446.05,11.05,X-LC,35.16,254817.0,25584.0,41714.0,-0.54,11.16,16.37,29.35,7.0,0.61,1.97,21.08,XY25,ATH,FMCG
55,NESTLEIND,2268.60,2755.00,-12.59,X-LC,26.37,277512.0,12086.0,44818.0,-0.57,4.55,16.15,21.44,10.0,0.27,2.14,11.03,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-0.4,M-LC,45.05,99755.0,21393.0,12639.0,-2.35,27.3,12.67,43.43,77.0,1.69,0.77,34.07,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,45.05,99755.0,21393.0,12639.0,-2.35,27.30,12.67,43.43,77.0,1.69,0.77,34.07,XY24,NTT,MISC
32,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,60.44,151260.0,347.0,60111.0,-0.94,0.23,39.74,40.06,100.0,0.01,1.17,23.72,AR,ATH,AUTO
17,CAMS,3643.00,5250.99,-1.11,H-SC,72.53,111275.0,9271.0,35753.0,-2.42,9.09,32.13,44.14,125.0,0.26,0.86,26.81,X40N,ATH,MISC
83,UNIONBANK,123.87,163.00,-4.91,M-LC,83.52,166229.0,25389.0,19100.0,-2.86,18.03,11.49,31.59,86.0,1.33,1.28,44.40,XY24,NTT,BANKS
90,WIPRO,243.46,420.00,-5.63,M-LC,48.35,161566.0,10621.0,98830.0,0.38,7.04,61.17,72.51,68.0,0.11,1.25,11.32,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,WHIRLPOOL,1167.49,2270.00,-32.52,M-SC,58.24,196804.0,25183.0,136877.0,-0.10,14.67,69.55,94.43,194.0,0.18,1.52,45.25,XR,NTT,DURABLES
90,WIPRO,243.46,420.00,-5.63,M-LC,48.35,161566.0,10621.0,98830.0,0.38,7.04,61.17,72.51,68.0,0.11,1.25,11.32,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-29.44,M-MC,64.84,186083.0,6275.0,106142.0,-1.45,3.49,57.04,62.52,170.0,0.06,1.44,33.91,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-49.97,H-SC,54.95,130380.0,7044.0,127785.0,-1.36,5.71,98.01,109.32,122.0,0.06,1.01,28.81,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,60.44,151260.0,347.0,60111.0,-0.94,0.23,39.74,40.06,100.0,0.01,1.17,23.72,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.0,-53.72,L-SC,96.70,68333.0,-45216.0,85266.0,-0.53,-39.82,124.78,35.27,260.0,-0.53,0.53,67.25,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.0,7483.33,M-SC,100.00,83224.0,-10586.0,87202.0,3.53,-11.28,104.78,81.67,238.0,-0.12,0.64,61.17,XR,NTT,CERAMICS
52,MASFIN,326.60,399.5,-23.67,H-SC,70.33,86985.0,-10995.0,32863.0,-1.39,-11.22,37.78,22.32,141.0,-0.33,0.67,26.16,XR,ATH,FINANCE
50,LICI,827.06,983.0,-5.02,H-LC,87.91,96562.0,11375.0,4683.0,-0.66,13.35,4.85,18.85,35.0,2.43,0.75,28.57,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.0,-0.40,M-LC,45.05,99755.0,21393.0,12639.0,-2.35,27.30,12.67,43.43,77.0,1.69,0.77,34.07,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7483.33,M-SC,100.0,83224.0,-10586.0,87202.0,3.53,-11.28,104.78,81.67,238.0,-0.12,0.64,61.17,XR,NTT,CERAMICS
85,VALIANTORG,512.64,838.00,-389.46,H-SC,98.9,123162.0,-10124.0,94724.0,-3.43,-7.60,76.91,63.47,142.0,-0.11,0.95,106.45,XR,NTT,CHEMICALS
46,JPPOWER,18.73,26.20,-33.95,L-SC,97.8,134269.0,-7573.0,64140.0,-0.00,-5.34,47.77,39.88,258.0,-0.12,1.04,36.07,XY24,NTT,POWER
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.7,68333.0,-45216.0,85266.0,-0.53,-39.82,124.78,35.27,260.0,-0.53,0.53,67.25,XR,NTT,HOTELS
35,HINDZINC,514.80,744.74,37.93,H-LC,95.6,103320.0,360.0,45626.0,-0.13,0.35,44.16,44.67,25.0,0.01,0.80,33.33,X5K,ATH,METALS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,43.23
2,50,71.99


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.88
XY25     14.29
X40      12.63
XR       11.05
OX40N     8.52
X40N      8.48
AR        8.10
X5K       2.36
X200      1.51
SR        1.21
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.64
X-LC    15.61
M-SC    14.42
H-LC    11.15
H-MC     9.01
M-LC     8.51
X-MC     7.70
M-MC     5.81
X-SC     2.82
L-SC     2.50
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.67,-3.03,37.46
IT,11.46,-11.86,68.65
BANKS,8.52,-10.27,54.71
MISC,6.58,-26.87,76.47
FINANCE,6.40,-21.26,67.92
PAINTS,5.64,-19.37,42.89
ELECTRICAL,5.16,-5.21,50.85
HEALTHCARE,4.76,-4.24,34.69
AUTO,4.70,-11.05,59.88


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2796291.0,25
XR,1130449.0,13
AR,1021188.0,9
X40,683847.0,9
XY25,537151.0,10
X40N,430854.0,8
OX40N,326515.0,11
SR,188013.0,2
X5K,97132.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1950267.0,19
M-SC,1458745.0,18
X-LC,641139.0,9
M-MC,592347.0,5
H-MC,559694.0,7
H-LC,512341.0,10
X-MC,419100.0,6
M-LC,340825.0,6
L-SC,274655.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,917153.0,6
M-SC,XY24,587350.0,5
H-SC,AR,522256.0,3
M-MC,XY24,397741.0,3
M-SC,XR,342081.0,4
X-LC,X40,324629.0,3
H-MC,XY24,227888.0,2
M-LC,XY24,223247.0,4
H-SC,XR,204773.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
